In [1]:
import glob, json, pandas as pd
df1 = pd.read_csv('../data/coco_degradation/val2017.csv')

df1.head(2)

,image,quality,width,height,channels,mean_squared_error,peak_signal_noise_ratio,structural_similarity
0,000000000139.jpg,1,640,426,3,454.463,21.556,0.600
1,000000000285.jpg,1,586,640,3,781.224,19.203,0.345


In [2]:
df1.channels.value_counts()

3    499000
1      1000
Name: channels, dtype: int64

In [3]:
def load(f):
    with open(f) as jsf:
        return json.load(jsf)

In [4]:
jsons = glob.glob('../results/*/filesizes*.json')
objs = [load(f) for f in jsons]

In [5]:
def dicts(objs):
    for obj in objs:
        q = obj['quality']
        if q == 'original' or q == 0:
            continue
        for img, size in obj['filesizes'].items():
            yield dict(
                quality=q,
                image=img,
                filesize=size,
            )

In [6]:
df2 = pd.DataFrame(dicts(objs))

In [7]:
# unique img<->Q
df2.drop('filesize', 1).value_counts().value_counts()

1    500000
dtype: int64

In [8]:
idf1 = df1.set_index(['image', 'quality'])
idf2 = df2.set_index(['image', 'quality'])

In [9]:
idf1['filesize'] = idf2.filesize

In [10]:
idf1.head(2)

,,width,height,channels,mean_squared_error,peak_signal_noise_ratio,structural_similarity,filesize
image,quality,,,,,,,
000000000139.jpg,1,640,426,3,454.463,21.556,0.600,4748
000000000285.jpg,1,586,640,3,781.224,19.203,0.345,10097


In [11]:
df = idf1.reset_index()
df.to_csv(
    '../data/val2017_degraded.csv',
    float_format='%.3f'
)

## Mean values per quality 

In [12]:
df['out_channels'] = df.channels

In [13]:
images_3_to_1 = pd.read_csv('../data/monoRGB_val2017.csv', index_col=0, quotechar="'")
images_3_to_1.head(2)

,image
0,000000001425.jpg
1,000000001490.jpg


In [14]:
idf = df.set_index('image')
idf.loc[images_3_to_1.image, 'out_channels'] = 1
idf['B_per_kpx'] = 1000 * idf.filesize / (idf.width * idf.height * idf.out_channels)
idf.head(2)

,quality,width,height,channels,mean_squared_error,peak_signal_noise_ratio,structural_similarity,filesize,out_channels,B_per_kpx
image,,,,,,,,,,
000000000139.jpg,1,640,426,3,454.463,21.556,0.600,4748,3,5.804969
000000000285.jpg,1,586,640,3,781.224,19.203,0.345,10097,3,8.974154


In [15]:
pd.options.display.max_rows = 4
mdf = idf.drop(['channels', 'out_channels', 'width', 'height'], 1).groupby('quality').mean()
mdf

,mean_squared_error,peak_signal_noise_ratio,structural_similarity,filesize,B_per_kpx
quality,,,,,
1,536.180397,21.204653,0.538857,7203.7820,9.290775
2,535.909813,21.206728,0.539011,7207.2246,9.295268
...,...,...,...,...,...
99,0.224027,54.850101,0.998937,215370.4628,273.442582
100,0.158625,56.272696,0.999161,257475.1808,325.305082


In [16]:
mdf.to_csv(
    '../data/val2017_means_per_Q.csv',
    float_format='%.3f'
)